In [62]:
import pandas as pd
opinions=pd.read_csv("amazon.txt",sep='\t', header=None, names=["Review","Result"],quoting=3)
opinions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  1000 non-null   object
 1   Result  1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [63]:
opinions['Review'].head()

0    So there is no way for me to plug it in here i...
1                          Good case, Excellent value.
2                               Great for the jawbone.
3    Tied to charger for conversations lasting more...
4                                    The mic is great.
Name: Review, dtype: object

In [69]:

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

import re
# connecting different instances of the word to one, like connection, connections, connective, connected
stemmer=SnowballStemmer('english')
# stop words are meaningless common words like and, a, an
stop_words=stopwords.words("english")
# applying lambda to each column: replacing every chaaracter that is not a letter to a blank, splitting into words,
# and we use stemmer if the word is not a stopword, and than we join it together into a sentence with blank
opinions["cleaned"]=opinions["Review"].apply(lambda x: " ".join(
    [stemmer.stem(i) for i in re.sub("[^a-zA-Z]"," ",x).split() if i not in stop_words]).lower())
opinions["cleaned"].head()

0              so way plug us unless i go convert
1                            good case excel valu
2                                    great jawbon
3    tie charger convers last minut major problem
4                                   the mic great
Name: cleaned, dtype: object

In [131]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

x_train,x_test,y_train,y_test=train_test_split(opinions['Review'],opinions["Result"],test_size=0.2)
pip=Pipeline([('vectorizer',TfidfVectorizer(ngram_range=(1, 2),stop_words="english")),
              ('classifier',LinearSVC(random_state=10))
             ])
opinions_model=pip.fit(x_train,y_train)
opinions_model.score(x_test,y_test)


0.79

In [132]:
from sklearn.model_selection import cross_val_score
y_train_pred=cross_val_score(opinions_model,x_train,y_train,cv=5)
y_train_pred

array([0.78125, 0.74375, 0.79375, 0.78125, 0.78125])

In [133]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_train_pred2=cross_val_predict(opinions_model,x_train,y_train,cv=5)
confusion_matrix(y_train,y_train_pred2)

array([[313,  90],
       [ 89, 308]], dtype=int64)

In [134]:
from sklearn.metrics import precision_score,recall_score
print(precision_score(y_train,y_train_pred2))
print(recall_score(y_train,y_train_pred2))


0.7738693467336684
0.7758186397984886


In [135]:
y_test_pred=cross_val_score(opinions_model,x_test,y_test,cv=5)
y_test_pred

array([0.7 , 0.6 , 0.85, 0.8 , 0.7 ])

In [121]:
from sklearn.ensemble import GradientBoostingClassifier
pip2=Pipeline([('vectorizer',TfidfVectorizer(ngram_range=(1, 2),stop_words="english",max_df=8)),
              ('classifier', GradientBoostingClassifier(random_state=10))
             ])
opinions_model_2=pip2.fit(x_train,y_train)
opinions_model_2.score(x_test,y_test)


0.655

In [122]:
y_test_pred_2=cross_val_score(opinions_model_2,x_test,y_test,cv=5)
y_test_pred_2

array([0.675, 0.7  , 0.625, 0.55 , 0.725])

In [123]:
y_test_pred_2=cross_val_predict(opinions_model_2,x_test,y_test,cv=5)
confusion_matrix(y_test,y_test_pred_2)

array([[86, 15],
       [54, 45]], dtype=int64)